
Задание для личного проекта, который нужно будет сдавать для оценки:
# Написать чат бота, который будет говорить погоду в двух городах - Москва и СПб
# Бот должен уметь здороваться, прощаться.
# Бот должен понимать такие даты, как: сейчас, сегодня, завтра.
# Попрощаться - значит закончить беседу (после прощания, он не должен писать никаких сообщений пока вы не напишите)
# Бот должен как-то реагировать на непонятные высказывания
# Базу погоды можно составить самим - на два дня, на два города или использовать API (например, OpenWeatherMap (https://openweathermap.org/)).
# Бот должен работать и продолжать вас что-то спрашивать пока вы не закончите беседу.
# Бот должен запрашивать всю необходимую информацию для прогноза (если не хватает какого-то слота (город или дата), он должен его уточнить)

Один из сценариев скрипта:
1. Написать функцию распознания интенции (Например, Natasha)
2. Функция для того, чтобы здороваться.
3. Функция для погоды
4. Функция, чтобы прощаться.
5. Функция для ошибок.

То есть, из фич только погода.

Мануал как делать бота на питоне

(https://thecode.media/python-bot/)Если что-то не получается, все падает с ошибкой и т.д. - смело пишите сюда или в личку.

Deadline по проекту: сессионная неделя.
ДЗ можно просто отправить, я посмотрю, и если что-то не так, то доработать.
Проект нужно будет показывать. Можно показать, и если что-то не так, то доделать.

P.S. Каждый член команды должен будет ответить на вопросы по боту и рассказать как это работает.

P.P.S Не забывайте про домашнее задание - это почти половина оценки

In [26]:
import json
import urllib3
import spacy
from secret import *
import csv
import os

from datetime import date
from datetime import datetime
import time
nlp = spacy.load('ru_core_news_md')

In [27]:
http = urllib3.PoolManager()

### geo

In [28]:
def place_to_geo(place, city, map_key = map_key):
    '''

    :param place: Name of place to search
    :param city: Name of city
    :param map_key: API KEY from https://developer.mapquest.com/
    :return: dict {'lat': '55.6280353', 'lon': '36.9952076310053'}
    '''
    geo = {}

    url = f'http://open.mapquestapi.com/nominatim/v1/search.php?key={map_key}&format=json&q={place}+{city}&addressdetails=1&limit=3'
    response = http.request("GET", url)
    data = json.loads(response.data)[0]

    geo['lat'] = data['lat']
    geo['lon'] = data['lon']
    return geo

In [29]:
geo_ll =  place_to_geo("Невский проспект", "Москва")
geo_ll

{'lat': '59.9316703', 'lon': '30.3588031'}

### weather

In [30]:
def location_to_weather(geo_ll, weather_key = weather_key):
    '''
    :param geo_ll: dict from place_to_geo
    :param weather_key: api key from https://openweathermap.org/api/one-call-api
    :return: dict with temp for 7 days
    '''
    weather = {}

    lat = geo_ll['lat']
    lon = geo_ll['lon']

    url = f'https://api.openweathermap.org/data/2.5/onecall?lat={lat}&lon={lon}&exclude=current,minutely,hourly,alerts&appid={weather_key}&units=metric'
    response = http.request("GET", url)
    data = json.loads(response.data)
    for day in data['daily']:
        weather[day['dt']]=  day['feels_like']
    return weather


In [31]:
weather = location_to_weather(geo_ll)
weather

{1640163600: {'day': -17.54, 'night': -19.82, 'eve': -16.64, 'morn': -17.04},
 1640250000: {'day': -12.54, 'night': -10.92, 'eve': -10.95, 'morn': -15.44},
 1640336400: {'day': -13.06, 'night': -23.05, 'eve': -13.61, 'morn': -7.66},
 1640422800: {'day': -18.75, 'night': -13.59, 'eve': -18.37, 'morn': -22.7},
 1640509200: {'day': -20.61, 'night': -21.95, 'eve': -19.63, 'morn': -12.22},
 1640595600: {'day': -11.56, 'night': -18.51, 'eve': -17.28, 'morn': -19.4},
 1640682000: {'day': -9.35, 'night': -11.9, 'eve': -11.1, 'morn': -14.45},
 1640772000: {'day': -16.68, 'night': -14.42, 'eve': -15.36, 'morn': -15.27}}

### part of speech

In [32]:
def text_to_location(text, nlp = nlp):
    '''
    :param text: string input
    :param nlp: nlp spacy language model
    :return: list of locations
    '''

    text_spacy = nlp(text)

    locations = []

    if len(text_spacy.ents) == 0:
        raise NameError("Нет локации")
    else:
        for ent in text_spacy.ents:
            if ent.label_ == 'LOC':
                locations.append(ent.lemma_)

    return(locations)

In [33]:
text_to_location("как дела в Саратове и томске")

['саратов', 'томск']

## date

In [34]:
def text_to_date(text, nlp = nlp):
    '''
    :param text: string input
    :param nlp: nlp spacy language model
    :return: relative index of a date
    '''

    today = time.mktime(date.today().timetuple())
    tokens = []
    today_unix = time.mktime(date.today().timetuple())

    text_spacy = nlp(text)
    for token in text_spacy:
        tokens.append(token.lemma_)

    if 'сегодня' in tokens:
        return 0
    elif 'завтра' in tokens:
        return 1
    elif 'сейчас' in tokens:
        return 0
    elif 'послезавтра' in tokens:
        return 2
    else:
        raise NameError('Плохо расслышал дату. Это сегодня, завтра или послезавтра?')

In [35]:
text_to_date("Скажи погоду в Санкт-Петербурге на сегодня")

0

## store text

In [78]:
def text_to_csv(text, user, timestamp, storage = "csv", file = "text_logs.csv"):
    with open(file, 'a', newline='', encoding="utf-8") as csvfile:
        logwriter = csv.writer(csvfile, delimiter=',',
                                quotechar='|', quoting=csv.QUOTE_MINIMAL)
        logwriter.writerow([text, user, timestamp])

In [79]:
ts = time.mktime(date.today().timetuple())
text_to_csv(text = "Сколько вешать москва",user =  "seva", timestamp = ts)

## drop logs

In [80]:
os.remove("text_logs.csv")

## text to answer

In [39]:
# text  = "Скажи погоду Москве на завтра"
text = "сколько вешать в граммах в Москве"


In [160]:
def message_to_reply(text, user = "test_user",
                     loggs = False):
    '''

    :param text: string with text
    :return: reply
    '''

    timestamp = time.time()

    try:
        text_to_csv(text, user, timestamp)
        if loggs:
            text = ""
    except NameError:
        return ("Ошибка логов при записи")
    if loggs:
        try:
            with open('text_logs.csv', newline='', encoding="utf-8") as csvfile:
                logreader = csv.reader(csvfile, delimiter=',', quotechar='|')
                for row in logreader:
                    if row[1] == user and row[2] != timestamp:
                        text = text + " " + row[0]
        except NameError:
            return ("Ошибка чтения")
    print(text)
    try:
        location = text_to_location(text)
    except NameError:
        return ("А для какого места тебе нужна погода?")
    try:
        date_index = text_to_date(text)
    except NameError:
        return ("Cкажи, а на какой день делать прогноз?")
    try:
        geo_ll = place_to_geo("", location)
    except NameError:
        return ("Попробуй другую локацию")
    try:
        weather = location_to_weather(geo_ll)
    except NameError:
        return (f'Не смог найти погоду для {location}')
    try:
        weather_day = weather[list(weather.keys())[date_index]]
        day = date.fromtimestamp(list(weather.keys())[date_index])
        return f'Погода в городе {location[0].title()} {day} ' \
               f'днём: {weather_day["day"]} ;' \
               f'вечером {weather_day["eve"]}'
    except NameError:
        return (f' ошибка в последнем действии')


In [161]:
text  = "Скажи погоду Москве на завтра"
# text = "сколько вешать в граммах "

message_to_reply(text, loggs = False)

Скажи погоду Москве на завтра


'Погода в городе Москва 2021-12-23 днём: -20.59 ;вечером -20.55'

## end conversation

In [162]:
os.remove("text_logs.csv")

## conversation with logs

In [163]:
text  = "Привет как дела"

message_to_reply(text)

Привет как дела


'А для какого места тебе нужна погода?'

In [164]:
text  = "Погода в Москве"

message_to_reply(text, loggs = True)

 Привет как дела Погода в Москве


'Cкажи, а на какой день делать прогноз?'

In [165]:
text = "на завтра"
message_to_reply(text, loggs = True)

 Привет как дела Погода в Москве на завтра


'Погода в городе Москва 2021-12-23 днём: -20.59 ;вечером -20.55'

In [166]:
os.remove("text_logs.csv")